## Intro

In this Notebook I will train and evaluate two different Trigram models.
More precisely I will use two different approaches for creating a Trigram model.
Approach 1 is to build a matrix that holds the true probability distribution of which character 
comes next given that the model has been given two particular preceeding characters.
We then sample from this distribution to produce the next character in the output 'name'.

The second approach is to use a Neural Net. We take in the two preceeding characters to generate the next one 
and so on. The goal is to achieve the same performance or loss using the Neural Net approach as with approach 1.
As you will see approach 1 is the perfect approach given the loss function I will use. 
It is impossible to achieve better loss with a Neural Net approach to the Trigram model.

In [1]:
import torch

## First approach

We begin by loading in all the names from the text file:

In [2]:
words = open('names_dataset.txt', 'r').read().splitlines()

A peak at some names

In [3]:
words[0:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

Splitting up the Dataset into a Train and Test split. 80/20

In [57]:
from sklearn.model_selection import train_test_split

# Split the data
train_names, temp_names = train_test_split(words, test_size=0.2, random_state=42)  # 20% for testing
val_names, test_names = train_test_split(temp_names, test_size=0.5, random_state=42)  # 20% for testing

print("Number of Training Names:", len(train_names))
print("Number of Validation Names:", len(val_names))
print("Number of Testing Names:", len(test_names))



Number of Training Names: 25626
Number of Validation Names: 3203
Number of Testing Names: 3204


Initializing the matrix that will hold the counts for future characters,
please observe that we need 256 rows because for 27 characters the possible
permutations of 2 characters following each other is 27^2 = 27 * 27 = 729.
But I will actually use 729 - 27 = 720 rows because I will ignore all entries
that has the * character as the last because if that has happened the model has already stopped producing output.

In [36]:
# Every row in this matrix will eventually be filled with the prob-distribution of the next characters given two particular preceeding characters
N = torch.zeros((702, 27), dtype=torch.int32)
N.shape

torch.Size([702, 27])

Creating two mappings between integers and characters so I can represent every characters with an integer in computations

In [37]:
chars = sorted(list(set(''.join(words))))
chars.insert(0, '*')
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
stoi
itos

{0: '*',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

Creating two mappings between pairs of characters and Integers. 
Ignoring all entries that end with *, because if that happens the model has already stopped producing output.

In [38]:
chars = sorted(list(set(''.join(words))))
chars.insert(0, '*')

twochars = []
for char1 in chars:
    for char2 in chars:
        if char2 != '*':
            twochars.append(f'{char1}{char2}')

doublettoi = {s:i for i, s in enumerate(twochars)}
itodoublet = {i:s for s,i in doublettoi.items()}


Time to populate our matrix with counts of future characters!
I will use * to signal beginning and end of string. This will also be treated as a character.

In [39]:
for w in train_names:
    chs = ['*'] + list(w) + ['*']
    for i in range(len(chs) - 2):
        doublet = chs[i] + chs[i + 1]
        ch2 = chs[i + 2]
        ix1 = doublettoi[doublet]
        ix2 = stoi[ch2]
        # print(f"{doublet}, {ch2}")
        N[ix1, ix2] += 1


Small regularization to N:

In [40]:
# Adding a tiny bit of smoothing to N so we dont get any zero-probabilities:
N += 1

This code will visualize our matrix that holds the counts. Darker blue indicates more instances and white indicates no instances.
Beware that the matrix plot is very long since we have so many permutations of two characters.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16, 420))
plt.imshow(N, cmap='Blues')
for i in range(702):
    for j in range(27):
        chstr = itodoublet[i] + ' ' + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray')
plt.axis('off')

Now we will convert this Count Matrix to a Probability Matrix:

In [41]:
P = (N).float()
P.shape

torch.Size([702, 27])

Time to convert the Count-Matrix to a probability-Matrix. Now every row will contain a probability distribution based on the counts of that row.

In [42]:
P /= P.sum(1, keepdim=True)

This code will visualize the probabilites for the next character given the previous two characters. Please beware that this matrix is very long so after you've had a look at the matrix maybe clear the ouput so you don't have to scroll a lot.

In [ ]:


import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16, 420))
plt.imshow(P, cmap='Blues')
for i in range(702):
    for j in range(27):
        chstr = itodoublet[i] + ' ' + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, "{:.3f}".format(P[i, j].item()), ha='center', va='top', color='gray')
plt.axis('off')

Before I will generate names I will find the probability distribution of the first character in all the names. This is because for our Trigram model to be able to start generating outputs we need two input characters. We only have '*' in the beginning so we need a first character also. We can generate a random first character but this will produce far more unnatural names rather than if we sample from the true distribution of the first characters in our dataset. Therefore we perform the following step: 

In [58]:
names = train_names

# Initializing a dictionary
first_char_counts = {}

# Counting each first character
for name in names:
    if name:
        first_char = name[0].lower()
        if first_char in first_char_counts:
            first_char_counts[first_char] += 1
        else:
            first_char_counts[first_char] = 1

# Converting counts to probabilities
total_names = len(names)
first_char_probabilities = {char: count / total_names for char, count in first_char_counts.items()}

# Plucking out the probabilities for each character
sorted_probs = sorted(first_char_probabilities.items())  
prob_values = [prob for char, prob in sorted_probs]  

# Converting to a PyTorch tensor
first_character_probs = torch.tensor(prob_values, dtype=torch.float32)

Now we can sample from P to get the next character until we get *, in that case we stop generating and we break.

In [48]:
# We can generate 20 names for example:

g = torch.Generator().manual_seed(423283494)

for i in range(50):
    out = []

    # Sampling the first character:
    first_char = torch.multinomial(first_character_probs, num_samples=1)
    first_char = itos[first_char.item()]
    if first_char == '*':
        continue
    first_doublet = f'*{first_char}'
    first_doublet_index = torch.tensor(doublettoi[first_doublet])
    out.append(first_char)
    first_doublet = f'*{first_char}'

    ix = doublettoi[first_doublet]
    
    while True:

        # Pluck out the probability distribution for the next character
        p = P[ix]

        #print('ix: ', ix)
        #print('current input doublet: ', itodoublet[ix])
        # Sampling
        next_char_ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()

        #print('next char ix: ', next_char_ix)
        if next_char_ix == 0:
            break
        #print('Adding to out: ', itos[next_char_ix])
        out.append(itos[next_char_ix])
        next_doublet = ''.join(out[-2:])
        #print('next_doublet: ', next_doublet)

        ix = doublettoi[next_doublet]

    print('Name: ', ''.join(out))

Name:  nueth
Name:  lesaillindynklyn
Name:  cam
Name:  od
Name:  xnnia
Name:  bana
Name:  imalanna
Name:  im
Name:  nin
Name:  azion
Name:  quin
Name:  gre
Name:  here
Name:  jayana
Name:  vis
Name:  rannogpjxjeryah
Name:  rayk
Name:  ceovfrbah
Name:  youray
Name:  q
Name:  kasha
Name:  iva
Name:  bentrukwu
Name:  iys
Name:  caironna
Name:  grosabiq
Name:  rumalie
Name:  jan
Name:  jerne
Name:  vannia
Name:  lan
Name:  ylendanni
Name:  chajay
Name:  hud
Name:  ivedsynnjaziesminluwen
Name:  gra
Name:  inglekileelyn
Name:  is
Name:  um
Name:  gian
Name:  em
Name:  stcjldmzbres
Name:  ka
Name:  selie
Name:  kencmla
Name:  aydne


Let's evaluate our Trigram model on our test-set and see what our Negative Log-likehihood is. We want this value (loss) to be as low as possible. A value of zero indicates that our model is 100% confident about every character in every word showing up when they actually do. In other words it would be able to perfectly predict the names in our dataset. This can't happen in practice but in theory that is kind of what we aim for.

In [49]:
import torch.nn.functional as F


log_likelihood = 0.0

cross_entropy_loss = 0.0

count = 0

for w in test_names:
    chs = ['*'] + list(w) + ['*']
    for i in range(len(chs) - 2):
        doublet = chs[i] + chs[i + 1]
        ch2 = chs[i + 2]
        ix1 = doublettoi[doublet]
        ix2 = stoi[ch2]

        # Used for Calculating The Log-Likelihood Loss:
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        
        count += 1


print(f"Log Likelihood is: {log_likelihood}")
nll = -log_likelihood
print(f"Negative Log Likelihood is: {nll}")
mean_nll = nll/count
print(f"Average Negative Log Likelihood is: {mean_nll}")

Log Likelihood is: -41790.421875
Negative Log Likelihood is: 41790.421875
Average Negative Log Likelihood is: 2.136197090148926


So the loss we achieve on the first approach using a probability matrix is roughly 2.126.

Now we will do the same thing but with a Neural Net instead:

In [60]:
# Creating a Neural Net to solve the same task:

# Creating the Train Dataset:

xs, ys = [], []

for w in train_names:
    chs = ['*'] + list(w) + ['*']
    for i in range(len(chs) - 2):
        doublet = chs[i] + chs[i + 1]
        ch2 = chs[i + 2]
        ix1 = doublettoi[doublet]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num_examples = xs.nelement()
print('Number of examples: ', num_examples)
xs, ys

Number of examples:  156871


(tensor([  8, 247, 367,  ..., 472, 141, 323]),
 tensor([14,  4,  9,  ..., 12, 12,  0]))

Our Neurons will each receive 702 inputs and we will have 27 neurons in total producing an output with dim 1 x 27. We will interpret these outputs as a probability distribution over the next character given an input of a two-character string.

In [114]:
# Initializing the Neural Network:

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 702), generator=g, requires_grad=True)

This is the code that will process inputs with our neural net, calculate the loss, from the loss calculate the gradients and from the gradient update the weights in our Neural Net properly. 

In the beginning I used one-hot encoding to encode my inputs to the Neural Net but later realized that that was equivalent to just plucking out a column from the matrix because every column of the weight matrix contains a certain probability ditribution for the next character given a certain input doublet.

In [138]:
# Performing Gradient Descent to improve the performance of the Network:
num_examples = xs.nelement()
print(num_examples)

for k in range(300):

    # Forward pass

    # I originally started with this method for encoding inputs that could be fed into the weight matrix.
    # However I realized that that was equal to plucking out a specific column from the weight matrix directly.
    # We encode our input with the one-hot encoding, creating a tensor that is filled with 701 zeros and 1 one to signal which of the possible
    # 702 input combinations is currently being fed to the network.
    # xenc = F.one_hot(xs_val, num_classes=702).float()
    
    logits = W[:, xs].T # Logits prediction
    counts = logits.exp() # Now we only have positive counts
    probs = counts / counts.sum(1, keepdim=True) # Now we have the probabilities outputted from the Neural Net for next character
    # Also adding a regularization term to the loss to prevent overfitting the train set:
    hyperparam_scalar = 0.05
    loss = -probs[torch.arange(num_examples), ys].log().mean() + hyperparam_scalar*abs(W).mean() # The loss we use here is the Negative Log-likelihood Loss
    print(loss.item())

    # Backward pass
    W.grad = None # Set to zero the gradient
    loss.backward()

    # Update
    W.data += -80 * W.grad



156871
2.504340171813965
2.498588800430298
2.4930167198181152
2.4876153469085693
2.4823765754699707
2.4772934913635254
2.4723575115203857
2.4675631523132324
2.4629037380218506
2.458373785018921
2.4539668560028076
2.4496774673461914
2.4455013275146484
2.4414334297180176
2.437469959259033
2.433605670928955
2.429837226867676
2.426161766052246
2.422574281692505
2.419071674346924
2.415651321411133
2.412309169769287
2.409043073654175
2.4058499336242676
2.4027271270751953
2.3996732234954834
2.396684169769287
2.3937597274780273
2.3908963203430176
2.3880927562713623
2.3853468894958496
2.3826558589935303
2.380019426345825
2.377434730529785
2.3749008178710938
2.372415542602539
2.369978427886963
2.3675875663757324
2.365241527557373
2.3629395961761475
2.360680103302002
2.358461618423462
2.3562824726104736
2.354142427444458
2.3520400524139404
2.3499741554260254
2.3479442596435547
2.345949411392212
2.3439879417419434
2.342060089111328
2.3401639461517334
2.338299512863159
2.336465358734131
2.334661006

I am going to evaluate the loss on the test set to decide what value of the hyperparameter is best when we train the network. We want it to generalize well of course.

In [67]:
# Creating the Val Dataset:

xs_val, ys_val = [], []

for w in val_names:
    chs = ['*'] + list(w) + ['*']
    for i in range(len(chs) - 2):
        doublet = chs[i] + chs[i + 1]
        ch2 = chs[i + 2]
        ix1 = doublettoi[doublet]
        ix2 = stoi[ch2]
        xs_val.append(ix1)
        ys_val.append(ix2)

xs_val = torch.tensor(xs_val)
ys_val = torch.tensor(ys_val)

num_examples = xs_val.nelement()
print('Number of examples: ', num_examples)

Number of examples:  19679


Let's evaluate the loss on the validation set:

In [88]:
# Performing Gradient Descent to improve the performance of the Network:
num_examples = xs_val.nelement()
print(num_examples)

logits = W[:, xs_val].T
counts = logits.exp() # Now we only have positive counts
probs = counts / counts.sum(1, keepdim=True) # Now we have the probabilities outputted from the Neural Net for next character
loss = -probs[torch.arange(num_examples), ys_val].log().mean() # The loss we use here is the Negative Log-likelihood Loss
print(loss.item())

19679
2.1695034503936768


After a couple hyperparameter trials I decided on 0.05 for the hyperparameter scalar. That seemed like a good value.

Now let's evaluate the Neural Network on the Test-set:

In [77]:
# Creating the Test Dataset:

xs_test, ys_test = [], []

for w in test_names:
    chs = ['*'] + list(w) + ['*']
    for i in range(len(chs) - 2):
        doublet = chs[i] + chs[i + 1]
        ch2 = chs[i + 2]
        ix1 = doublettoi[doublet]
        ix2 = stoi[ch2]
        xs_test.append(ix1)
        ys_test.append(ix2)

xs_test = torch.tensor(xs_test)
ys_test = torch.tensor(ys_test)

num_examples = xs_test.nelement()
print('Number of examples: ', num_examples)

Number of examples:  19563


In [143]:
import torch.nn.functional as F

# Performing Gradient Descent to improve the performance of the Network:
num_examples = xs_test.nelement()
print(num_examples)

logits = W[:, xs_test].T # Logits prediction
counts = logits.exp() # Now we only have positive counts
probs = counts / counts.sum(1, keepdim=True) # Now we have the probabilities outputted from the Neural Net for next character
loss = -probs[torch.arange(num_examples), ys_test].log().mean() # The loss we use here is the Negative Log-likelihood Loss

# Also Calculating the Cross-Entropy Loss for fun
# I will treat the probabilities from the probability matrix P as the true distribution of future characters.
# The cross_entropy loss function in pytorch takes in logits in the inputs arguments instead of probabilities so we can 
# send our logits directly in into the function.
CE_loss = F.cross_entropy(logits, P[xs_test, :])

print("Negative Log-Likelihood Loss ", loss.item())
print("Cross Entropy Loss: ", CE_loss.item())

19563
Negative Log-Likelihood Loss  2.1820690631866455
Cross Entropy Loss:  2.2577147483825684


If you run the training code for a couple of times you can actually achieve lower loss on the test-set than approach 1. This is actually really interesting and I dont know quite why at this point. I did not think that this was possible actually. But my best loss for approach 1 was: 2.156 and loss for approach 2 was: 2.152.

Now we can simulate some names with approach 2 as well:

In [92]:
for i in range(20):


    first_char = torch.multinomial(first_character_probs, num_samples=1)
    first_char = itos[first_char.item()+1]
    first_doublet = f'*{first_char}'
    first_doublet_index = torch.tensor(doublettoi[first_doublet])

    start_symbol = F.one_hot(first_doublet_index, num_classes=702).float() # Inputting start symbol to the Neural Network to make it produce output characters
    current_doublet = start_symbol
    # Initializing current name:
    current_word = []
    # Adding the first randomly generated character:
    current_word.append(first_char)

    # This variable will keep track of the index of the current input doublet:
    next_doublet_index = torch.tensor(1)

    while(True):
        logits = current_doublet @ W.T # Logits prediction
        counts = logits.exp() # Now we only have positive counts
        probs = counts / counts.sum(0, keepdim=True)
        # probs = torch.ones(27) / 27 This line is for anyone curious about the difference between a Trigram and random generation

        # Sampling next character index:
        next_single_char_index = torch.multinomial(probs, num_samples=1)
        # If it is * we have reached the end of the name
        if next_single_char_index == 0:
            break

        # Plucking out next character from index:
        next_char = itos[next_single_char_index.item()]
        # Appending to current name:
        current_word.append(next_char)
        # Plucking out next doublet input:
        next_doublet = current_word[-2:]
        next_doublet = ''.join(next_doublet)
        next_doublet_index = torch.tensor(doublettoi[next_doublet])
        current_doublet = F.one_hot(next_doublet_index, num_classes=702).float()

    # Printing the current name:
    current_word = ''.join(current_word)
    print(current_word)

kauriah
shon
da
zy
brissaniquellie
noore
kaymiereennesnblaimyah
aliseri
kaljwdhyleiggpjcni
aidi
mes
fin
ma
zarralani
penangzwparvpfqxafadyn
ittejah
ley
cregane
loah
sameigh


## That's that!
Btw I'm really interested in business and AI.
For any business ideas please reach out to filiplarssonnnn@gmail.com

Bye!